# HW3: Distance Geometry, Clustering, and Nearest Neighbors

In this assignment, you will explore how different learning methods
view the same data through the lens of distance and geometry.

You will work with the UCI Wine dataset and build a short visualization
story connecting data geometry, prediction, and dimensionality.

Instructions:
- Run this notebook top to bottom.
- Complete all TODO blocks.
- All figures must be generated by your own code.
- Clearly label axes and include short interpretations in markdown cells.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import pairwise_distances, accuracy_score
from scipy.spatial.distance import cdist
from sklearn_extra.cluster import KMedoids

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

## Load and Prepare the Data

In [ ]:
wine = load_wine(as_frame=True)
X = wine.data
y = wine.target

# Inspect X and y
print("Dataset Shape:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print("\nFeature names:")
print(X.columns.tolist())
print("\nClass distribution:")
print(y.value_counts().sort_index())
print("\nBasic statistics:")
print(X.describe())

## Task 1: Data Familiarization

Before fitting any models, explore the dataset.

In [ ]:
# Exploratory Plot 1: Class Balance
fig, ax = plt.subplots(figsize=(8, 5))
class_counts = y.value_counts().sort_index()
bars = ax.bar(class_counts.index, class_counts.values, color=['#ff7f0e', '#2ca02c', '#1f77b4'])
ax.set_xlabel('Wine Cultivar', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Class Distribution in Wine Dataset', fontsize=14, fontweight='bold')
ax.set_xticks([0, 1, 2])
ax.set_xticklabels(['Class 0', 'Class 1', 'Class 2'])

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Exploratory Plot 2: Feature Distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
features_to_plot = ['alcohol', 'color_intensity', 'proline', 'flavanoids']

for idx, feature in enumerate(features_to_plot):
    ax = axes[idx // 2, idx % 2]
    for class_label in [0, 1, 2]:
        data = X[y == class_label][feature]
        ax.hist(data, bins=20, alpha=0.6, label=f'Class {class_label}')
    
    ax.set_xlabel(feature.replace('_', ' ').title(), fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(f'Distribution of {feature.replace("_", " ").title()}', fontsize=12, fontweight='bold')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Exploratory Plot 3: Scatter plot of two features
fig, ax = plt.subplots(figsize=(10, 7))
colors = ['#ff7f0e', '#2ca02c', '#1f77b4']

for class_label in [0, 1, 2]:
    mask = y == class_label
    ax.scatter(X[mask]['alcohol'], X[mask]['color_intensity'], 
               c=colors[class_label], label=f'Class {class_label}', 
               alpha=0.7, s=60, edgecolors='black', linewidth=0.5)

ax.set_xlabel('Alcohol Content', fontsize=12)
ax.set_ylabel('Color Intensity', fontsize=12)
ax.set_title('Wine Cultivars: Alcohol vs Color Intensity', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Interpretation of Exploratory Analysis

**Key Observations:**

1. **Class Balance**: The dataset has a moderate class imbalance. Class 1 has the most samples (71), followed by Class 0 (59) and Class 2 (48). This is relatively balanced compared to many real-world datasets.

2. **Feature Distributions**: Different features show varying levels of class separability:
   - **Alcohol**: Shows good separation, with Class 2 having notably higher alcohol content
   - **Color Intensity**: Classes overlap but show different central tendencies
   - **Proline**: Exhibits strong separation, especially for Class 0
   - **Flavanoids**: Clear differences between classes, particularly Class 2

3. **2D Scatter Plot**: The alcohol vs color intensity plot reveals that classes are reasonably separable in this 2D space, though some overlap exists. This suggests distance-based methods may work well.

4. **Scale Differences**: Features have very different scales (e.g., alcohol ranges 11-15 while proline ranges 200-1600), making standardization essential for distance-based methods.

## Standardization

Distance based methods are sensitive to scale.

In [ ]:
# Standardize X
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

print("Standardized data statistics:")
print(X_scaled.describe())
print("\nMean of each feature (should be ~0):")
print(X_scaled.mean())
print("\nStd of each feature (should be ~1):")
print(X_scaled.std())

## Task 2: Geometry Through Clustering

In this task, you will study how clustering methods impose geometric structure.

In [ ]:
# Select 2 features for visualization
features = ['alcohol', 'color_intensity']
X2 = X_scaled[features].values

# K-means with different K values
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
k_values = [2, 3, 5]

for idx, k in enumerate(k_values):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = km.fit_predict(X2)
    
    ax = axes[idx]
    # Plot points
    scatter = ax.scatter(X2[:, 0], X2[:, 1], c=clusters, 
                        cmap='viridis', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    # Plot centroids
    ax.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], 
              c='red', marker='X', s=200, edgecolors='black', linewidth=2, 
              label='Centroids')
    
    ax.set_xlabel('Alcohol (standardized)', fontsize=11)
    ax.set_ylabel('Color Intensity (standardized)', fontsize=11)
    ax.set_title(f'K-Means with K={k}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# K-means with different initializations (K=3)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, seed in enumerate([42, 100, 200]):
    km = KMeans(n_clusters=3, random_state=seed, n_init=1)
    clusters = km.fit_predict(X2)
    
    ax = axes[idx]
    scatter = ax.scatter(X2[:, 0], X2[:, 1], c=clusters, 
                        cmap='viridis', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    ax.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], 
              c='red', marker='X', s=200, edgecolors='black', linewidth=2, 
              label='Centroids')
    
    ax.set_xlabel('Alcohol (standardized)', fontsize=11)
    ax.set_ylabel('Color Intensity (standardized)', fontsize=11)
    ax.set_title(f'K-Means (seed={seed})', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# K-medoids comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
k_values = [2, 3, 5]

for idx, k in enumerate(k_values):
    kmedoids = KMedoids(n_clusters=k, random_state=42)
    clusters = kmedoids.fit_predict(X2)
    
    ax = axes[idx]
    scatter = ax.scatter(X2[:, 0], X2[:, 1], c=clusters, 
                        cmap='plasma', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    # Plot medoids
    ax.scatter(kmedoids.cluster_centers_[:, 0], kmedoids.cluster_centers_[:, 1], 
              c='darkblue', marker='D', s=200, edgecolors='black', linewidth=2, 
              label='Medoids')
    
    ax.set_xlabel('Alcohol (standardized)', fontsize=11)
    ax.set_ylabel('Color Intensity (standardized)', fontsize=11)
    ax.set_title(f'K-Medoids with K={k}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Direct comparison: K-means vs K-medoids (K=3)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# K-means
km = KMeans(n_clusters=3, random_state=42)
km_clusters = km.fit_predict(X2)
axes[0].scatter(X2[:, 0], X2[:, 1], c=km_clusters, 
                cmap='viridis', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[0].scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], 
                c='red', marker='X', s=200, edgecolors='black', linewidth=2, label='Centroids')
axes[0].set_xlabel('Alcohol (standardized)', fontsize=11)
axes[0].set_ylabel('Color Intensity (standardized)', fontsize=11)
axes[0].set_title('K-Means (K=3)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# K-medoids
kmedoids = KMedoids(n_clusters=3, random_state=42)
km_clusters = kmedoids.fit_predict(X2)
axes[1].scatter(X2[:, 0], X2[:, 1], c=km_clusters, 
                cmap='viridis', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[1].scatter(kmedoids.cluster_centers_[:, 0], kmedoids.cluster_centers_[:, 1], 
                c='darkblue', marker='D', s=200, edgecolors='black', linewidth=2, label='Medoids')
axes[1].set_xlabel('Alcohol (standardized)', fontsize=11)
axes[1].set_ylabel('Color Intensity (standardized)', fontsize=11)
axes[1].set_title('K-Medoids (K=3)', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Interpretation of Clustering Results

**K-Means Observations:**
- **Effect of K**: As K increases, clusters become more granular. K=2 creates broad regions, K=3 aligns well with the visual structure, and K=5 subdivides the space further.
- **Initialization**: Different random seeds can lead to slightly different cluster assignments, though with multiple initializations (n_init=10), K-means typically converges to similar solutions.
- **Centroid positioning**: Centroids lie at the geometric mean of cluster points, which may not correspond to actual data points.

**K-Medoids Observations:**
- **Representatives**: Unlike K-means, medoids are actual data points, making them more interpretable as "prototype" examples.
- **Robustness**: K-medoids is less sensitive to outliers since it minimizes absolute distances rather than squared distances.
- **Similar structure**: For this dataset, K-means and K-medoids produce similar clustering patterns, suggesting the data doesn't have extreme outliers.

**Key Insight**: Both methods partition the space based on distance, but they represent clusters differently—K-means uses theoretical centroids while K-medoids uses actual exemplar points.

## Task 3: Prototype Methods for Classification

Now treat the wine cultivar as a class label.

In [ ]:
# k-NN with different k values
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
k_values = [1, 3, 5, 10, 20, 50]

# Create meshgrid for decision boundary
h = 0.02  # step size in the mesh
x_min, x_max = X2[:, 0].min() - 0.5, X2[:, 0].max() + 0.5
y_min, y_max = X2[:, 1].min() - 0.5, X2[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

for idx, k in enumerate(k_values):
    ax = axes[idx // 3, idx % 3]
    
    # Train k-NN
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X2, y)
    
    # Predict on mesh
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot decision boundary
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
    
    # Plot training points
    scatter = ax.scatter(X2[:, 0], X2[:, 1], c=y, 
                        cmap='viridis', edgecolors='black', s=50, linewidth=0.5)
    
    # Calculate accuracy
    acc = accuracy_score(y, knn.predict(X2))
    
    ax.set_xlabel('Alcohol (standardized)', fontsize=10)
    ax.set_ylabel('Color Intensity (standardized)', fontsize=10)
    ax.set_title(f'k-NN (k={k})\nTraining Accuracy: {acc:.3f}', 
                fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Prototype-based classifier using K-means per class
def prototype_classifier(X_train, y_train, X_test, n_prototypes_per_class=3):
    """
    Build a prototype-based classifier using K-means for each class.
    For each class, find k prototypes, then classify test points based on 
    nearest prototype.
    """
    prototypes = []
    prototype_labels = []
    
    # Find prototypes for each class
    for class_label in np.unique(y_train):
        X_class = X_train[y_train == class_label]
        if len(X_class) >= n_prototypes_per_class:
            km = KMeans(n_clusters=n_prototypes_per_class, random_state=42)
            km.fit(X_class)
            prototypes.extend(km.cluster_centers_)
            prototype_labels.extend([class_label] * n_prototypes_per_class)
    
    prototypes = np.array(prototypes)
    prototype_labels = np.array(prototype_labels)
    
    # Classify test points
    distances = cdist(X_test, prototypes)
    nearest_prototype = np.argmin(distances, axis=1)
    predictions = prototype_labels[nearest_prototype]
    
    return predictions, prototypes, prototype_labels

# Compare prototype methods with different numbers of prototypes
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
n_prototypes_list = [1, 2, 5]

for idx, n_proto in enumerate(n_prototypes_list):
    ax = axes[idx]
    
    # Get predictions and prototypes
    y_pred, prototypes, proto_labels = prototype_classifier(X2, y, X2, n_proto)
    
    # Create decision boundary
    Z_pred, _, _ = prototype_classifier(X2, y, np.c_[xx.ravel(), yy.ravel()], n_proto)
    Z_pred = Z_pred.reshape(xx.shape)
    
    # Plot decision boundary
    ax.contourf(xx, yy, Z_pred, alpha=0.3, cmap='viridis')
    
    # Plot training points
    ax.scatter(X2[:, 0], X2[:, 1], c=y, 
              cmap='viridis', edgecolors='black', s=50, linewidth=0.5, alpha=0.7)
    
    # Plot prototypes
    for class_label in np.unique(proto_labels):
        class_prototypes = prototypes[proto_labels == class_label]
        ax.scatter(class_prototypes[:, 0], class_prototypes[:, 1], 
                  c=f'C{class_label}', marker='*', s=500, 
                  edgecolors='black', linewidth=2, label=f'Class {class_label} prototypes')
    
    acc = accuracy_score(y, y_pred)
    
    ax.set_xlabel('Alcohol (standardized)', fontsize=11)
    ax.set_ylabel('Color Intensity (standardized)', fontsize=11)
    ax.set_title(f'Prototype Classifier ({n_proto} per class)\nAccuracy: {acc:.3f}', 
                fontsize=12, fontweight='bold')
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare k-NN (local) vs Prototype (global) behavior
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# k-NN with k=5 (local)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X2, y)
Z_knn = knn.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

axes[0].contourf(xx, yy, Z_knn, alpha=0.3, cmap='viridis')
axes[0].scatter(X2[:, 0], X2[:, 1], c=y, 
               cmap='viridis', edgecolors='black', s=50, linewidth=0.5)
axes[0].set_xlabel('Alcohol (standardized)', fontsize=11)
axes[0].set_ylabel('Color Intensity (standardized)', fontsize=11)
axes[0].set_title('k-NN (k=5): Local Decision Boundaries', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Prototype classifier (global)
y_pred, prototypes, proto_labels = prototype_classifier(X2, y, np.c_[xx.ravel(), yy.ravel()], 2)
Z_proto = y_pred.reshape(xx.shape)

axes[1].contourf(xx, yy, Z_proto, alpha=0.3, cmap='viridis')
axes[1].scatter(X2[:, 0], X2[:, 1], c=y, 
               cmap='viridis', edgecolors='black', s=50, linewidth=0.5, alpha=0.7)

# Get prototypes for plotting
_, prototypes, proto_labels = prototype_classifier(X2, y, X2, 2)
for class_label in np.unique(proto_labels):
    class_prototypes = prototypes[proto_labels == class_label]
    axes[1].scatter(class_prototypes[:, 0], class_prototypes[:, 1], 
                   c=f'C{class_label}', marker='*', s=500, 
                   edgecolors='black', linewidth=2)

axes[1].set_xlabel('Alcohol (standardized)', fontsize=11)
axes[1].set_ylabel('Color Intensity (standardized)', fontsize=11)
axes[1].set_title('Prototype Classifier: Global Voronoi Regions', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Interpretation of Classification Results

**k-NN Behavior:**
- **Small k (k=1)**: Decision boundaries are highly local and complex, following individual points exactly. This leads to perfect training accuracy (overfitting) but irregular decision regions.
- **Medium k (k=5-10)**: Balances local and global structure, creating smoother boundaries while still adapting to local patterns.
- **Large k (k=50)**: Boundaries become very smooth and global, potentially underfitting as local variations are ignored.

**Prototype-Based Classifier:**
- **Few prototypes (1 per class)**: Creates simple Voronoi tessellation with linear boundaries between class prototypes. Very global and simple.
- **More prototypes (5 per class)**: Allows for more complex, non-linear decision boundaries while maintaining interpretability.
- **Global structure**: Unlike k-NN, prototype methods impose a global geometric structure based on distances to a fixed set of representatives.

**Local vs Global Comparison:**
- **k-NN is adaptive**: Decision boundaries curve around individual points, capturing local structure.
- **Prototypes are global**: Decision regions are Voronoi cells determined by a small set of representative points.
- **Trade-off**: k-NN provides flexibility but can be unstable; prototypes are stable but less flexible.

**Key Insight**: k-NN makes predictions based on local neighborhoods, while prototype methods create global partitions. The choice depends on whether you believe class structure is better captured locally or globally.

## Task 4: When Distance Becomes Fragile

Investigate how distance based intuition changes as dimension increases.

In [ ]:
# Experiment: Distance versus dimension
n_features = X_scaled.shape[1]
dimensions = list(range(1, n_features + 1))
mean_nearest_distances = []
std_nearest_distances = []
max_min_ratios = []

for p in dimensions:
    # Use first p features
    X_p = X_scaled.iloc[:, :p].values
    
    # Compute pairwise distances
    distances = pairwise_distances(X_p)
    
    # For each point, find distance to nearest neighbor
    nearest_distances = []
    for i in range(len(distances)):
        # Get distances to all other points (exclude self)
        other_distances = distances[i][distances[i] > 0]
        if len(other_distances) > 0:
            nearest_distances.append(np.min(other_distances))
    
    mean_nearest_distances.append(np.mean(nearest_distances))
    std_nearest_distances.append(np.std(nearest_distances))
    
    # Also compute max/min ratio of nearest neighbor distances
    if len(nearest_distances) > 0:
        max_min_ratios.append(np.max(nearest_distances) / np.min(nearest_distances))

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Mean nearest neighbor distance vs dimension
axes[0].plot(dimensions, mean_nearest_distances, 'o-', linewidth=2, markersize=6)
axes[0].fill_between(dimensions, 
                      np.array(mean_nearest_distances) - np.array(std_nearest_distances),
                      np.array(mean_nearest_distances) + np.array(std_nearest_distances),
                      alpha=0.3)
axes[0].set_xlabel('Number of Dimensions', fontsize=12)
axes[0].set_ylabel('Mean Nearest Neighbor Distance', fontsize=12)
axes[0].set_title('Distance Growth with Dimensionality', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Plot 2: Max/min ratio (relative concentration)
axes[1].plot(dimensions, max_min_ratios, 's-', color='red', linewidth=2, markersize=6)
axes[1].set_xlabel('Number of Dimensions', fontsize=12)
axes[1].set_ylabel('Max/Min Nearest Distance Ratio', fontsize=12)
axes[1].set_title('Distance Concentration Effect', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# k-NN performance versus dimension
dimensions = list(range(1, n_features + 1))
knn_accuracies = {k: [] for k in [1, 3, 5, 10, 20]}

for p in dimensions:
    X_p = X_scaled.iloc[:, :p].values
    
    for k in knn_accuracies.keys():
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_p, y)
        acc = accuracy_score(y, knn.predict(X_p))
        knn_accuracies[k].append(acc)

# Plot k-NN accuracy vs dimension
fig, ax = plt.subplots(figsize=(12, 6))

for k, accuracies in knn_accuracies.items():
    ax.plot(dimensions, accuracies, 'o-', label=f'k={k}', linewidth=2, markersize=5)

ax.set_xlabel('Number of Dimensions', fontsize=12)
ax.set_ylabel('Training Accuracy', fontsize=12)
ax.set_title('k-NN Performance vs Dimensionality', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim([0.5, 1.05])

plt.tight_layout()
plt.show()

In [ ]:
# Distribution of pairwise distances in different dimensions
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
dims_to_plot = [1, 2, 3, 5, 8, 13]

for idx, p in enumerate(dims_to_plot):
    ax = axes[idx // 3, idx % 3]
    
    X_p = X_scaled.iloc[:, :p].values
    distances = pairwise_distances(X_p)
    
    # Get upper triangle (exclude diagonal)
    upper_triangle_indices = np.triu_indices_from(distances, k=1)
    all_distances = distances[upper_triangle_indices]
    
    ax.hist(all_distances, bins=50, alpha=0.7, edgecolor='black')
    ax.axvline(np.mean(all_distances), color='red', linestyle='--', 
              linewidth=2, label=f'Mean={np.mean(all_distances):.2f}')
    ax.set_xlabel('Pairwise Distance', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.set_title(f'{p}D: std/mean = {np.std(all_distances)/np.mean(all_distances):.3f}', 
                fontsize=11, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Interpretation: The Curse of Dimensionality

**Distance Growth:**
- As dimensions increase, the mean nearest neighbor distance grows approximately as √d (square root of dimension).
- This makes points increasingly far apart in absolute terms, even though they may be "close" in some dimensions.

**Distance Concentration:**
- The max/min ratio decreases as dimension grows, meaning all pairwise distances become more similar.
- In high dimensions, the nearest and farthest neighbors become almost equidistant from any given point.
- This is visualized in the histograms: distributions become tighter (lower std/mean ratio) and more concentrated around the mean.

**Impact on k-NN:**
- **Small k values** (k=1, 3) show high training accuracy initially but can become less reliable as all points seem equally distant.
- **Larger k values** (k=10, 20) show more stable behavior across dimensions but may miss local structure.
- Beyond ~8-10 dimensions, additional features provide diminishing returns for classification.

**Key Insight - The Curse of Dimensionality:**
In high-dimensional spaces:
1. Distances lose discriminative power (everything is far from everything)
2. The concept of "nearest neighbor" becomes less meaningful
3. Volume concentrates in corners and shells, not near the center
4. More data is needed to maintain the same density

This fundamentally challenges distance-based methods and motivates dimensionality reduction techniques.

## Final Visualization Story

### Research Question
**How do distance-based learning methods understand and partition data, and when do geometric intuitions break down?**

### The Journey Through Four Tasks

#### 1. **Discovery: The Data Has Structure** (Task 1)
The wine dataset revealed three distinct cultivars with measurable differences in chemical properties. Features like alcohol content, color intensity, and proline showed clear separation between classes. The 2D scatter plot demonstrated that even in low dimensions, classes occupy different regions of the feature space—suggesting that distance-based methods should work well.

**Key Finding**: The data is naturally separable, making it a good candidate for geometric methods.

---

#### 2. **Imposing Structure: How Algorithms See Geometry** (Task 2)
K-means and K-medoids revealed how clustering algorithms impose geometric structure on data:

- **K-means** creates Voronoi cells around theoretical centroids, minimizing squared distances
- **K-medoids** uses actual data points as representatives, creating more interpretable partitions
- Both methods assume spherical clusters and are sensitive to the choice of K

The comparison showed that while both methods produce similar clusterings for this dataset, they represent fundamentally different philosophies: theoretical optima (centroids) versus concrete exemplars (medoids).

**Key Finding**: Clustering methods partition space based on distance, but the choice of representative (centroid vs medoid) affects interpretability.

---

#### 3. **Local vs Global: Two Ways to Classify** (Task 3)
The classification experiments highlighted a crucial distinction:

- **k-NN is adaptive and local**: It creates complex, data-driven boundaries by looking at nearby points. Small k leads to overfitting (following every point), while large k creates overly smooth boundaries.

- **Prototype classifiers are global**: They partition the entire space based on a fixed set of representatives. This creates simple, interpretable Voronoi regions but may miss local variations.

The side-by-side comparison revealed that k-NN boundaries curve around individual points (local adaptation), while prototype boundaries are straight lines between representative points (global tessellation).

**Key Finding**: There's a fundamental trade-off between local flexibility (k-NN) and global stability (prototypes).

---

#### 4. **When Geometry Fails: The Curse of Dimensionality** (Task 4)
The final experiments revealed a profound limitation of distance-based methods:

- **Distance concentration**: As dimensions increase, all points become approximately equidistant from each other. The ratio of farthest to nearest neighbor approaches 1.

- **Loss of contrast**: In high dimensions, the histogram of pairwise distances becomes increasingly narrow (low std/mean ratio), meaning distances lose their discriminative power.

- **Practical impact**: k-NN performance plateaus or even degrades beyond 8-10 dimensions, as the notion of "nearest" becomes meaningless.

This phenomenon—the curse of dimensionality—explains why raw distance-based methods struggle in high-dimensional spaces and motivates techniques like dimensionality reduction, feature selection, and learned distance metrics.

**Key Finding**: In high dimensions, geometric intuition breaks down. Distance becomes a poor measure of similarity, and all points appear equally (dis)similar.

---

### Evolution of Understanding

My understanding evolved from:
1. **Optimism** (Task 1): "The data has clear structure; distance methods should work!"
2. **Appreciation** (Tasks 2-3): "Different distance-based methods offer different geometric perspectives—local vs global, theoretical vs concrete."
3. **Caution** (Task 4): "But all distance-based methods share a fundamental vulnerability: they fail when dimensions grow too large."

### Conclusion

Distance-based methods are powerful and interpretable when:
- Dimensionality is low (≤10 features)
- Features are appropriately scaled
- The geometric structure aligns with the problem

But they require careful handling in high dimensions, where:
- Distances concentrate and lose meaning
- More sophisticated approaches (kernel methods, manifold learning, deep learning) may be needed
- Feature engineering and dimensionality reduction become critical

The geometry of distance is both elegant and fragile—it works beautifully in the spaces we can visualize but becomes treacherous in the high-dimensional spaces where modern machine learning operates.